# The Electric Company!

The following Power Distribution network
<img src="elecpowernet.png" alt="Drawing" style="height: 300px;"/>
represents an electrical power distribution network connecting power generating points with power consuming points.

The arcs are undirected; that is, power may flow in either direction.  Points 1,4 and 7 are generating points with generating capacities and unit costs given below:

|Characteristic                           |1   |4   |7   |
|-----------------------------------------|----|----|----|
|Capacity (in thousands of kilowatt hours)|100 |20  |80  |
|Unit cost (\\$ per thousand KWH)         |15.0|13.5|21.0|
      
Points 2, 5 and 6 are consuming points with demands of 35,000, 50,000, and 60,000 KWH respectively.  There is no upper bound on transmission line capacity and the unit cost of transmission on each line segment is \\$11.00 per 1000 KWH.

### Set up and solve this problem to minimize the cost of meeting demand as a minimum cost network flow (linear program) in GAMS.  Make sure you have directed arcs in your formulation and the sum of the divergences is zero.

In [1]:
# Load the gams extension
%load_ext gams_magic

__The GAMS Model:__

In [3]:
%%gams

Set nodes /s, 1*7/;

Parameter demand(nodes) /s -145000, 2 35000, 5 50000, 6 60000/;
Parameter cost(nodes, nodes) /
        s.7 21, s.1 15, s.4 13.5, 1.2 11, 2.1 11, 2.3 11, 3.2 11,
        3.5 11, 5.3 11, 3.4 11, 4.3 11, 4.5 11, 5.4 11, 2.7 11, 7.2 11, 7.6 11, 6.7 11 /;

Set arcs(nodes, nodes);

Alias(nodes, i, j, k);
Positive Variable x(nodes, nodes);
Variable totalCost;

arcs(i, j) = yes$(cost(i, j) gt 0);
display Arcs;

x.UP("s","1") = 100000;
x.UP("s","4") = 20000;
x.UP("s","7") = 80000;

Equations balance(nodes), obj;
balance(k).. sum(i$arcs(i,k), x(i,k)) - sum(j$arcs(k,j), x(k,j)) =e= demand(k);
obj.. totalCost =e= sum((i,j)$arcs(i,j), cost(i,j)*x(i,j));

Model power /all/;
Solve power using lp minimizing totalCost;

display x.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),4760000.0,9,18,LP,CPLEX,0


__Pull the solution from GAMS and print just the variable values that are needed:__

In [4]:
%gams_pull -d x

x = x.drop(columns=['marginal','lower','upper','scale'])
x.columns = ['from','to','value']
display(x.loc[(x['to'] != 'dummy') & (x['value'] > 0)])

,from,to,value
0,s,1,65000.0
1,s,4,20000.0
2,s,7,60000.0
3,1,2,65000.0
5,2,3,30000.0
9,3,5,30000.0
11,4,5,20000.0
16,7,6,60000.0
